<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Title" data-toc-modified-id="Title-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Title</a></span><ul class="toc-item"><li><span><a href="#Business-Understanding-and-Business-Problem" data-toc-modified-id="Business-Understanding-and-Business-Problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Business Understanding and Business Problem</a></span><ul class="toc-item"><li><span><a href="#Cost-of-Errors" data-toc-modified-id="Cost-of-Errors-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Cost of Errors</a></span></li></ul></li><li><span><a href="#Data-Understanding" data-toc-modified-id="Data-Understanding-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Understanding</a></span><ul class="toc-item"><li><span><a href="#Initial-Data" data-toc-modified-id="Initial-Data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Initial Data</a></span></li></ul></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Final-Model" data-toc-modified-id="Final-Model-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Final Model</a></span></li><li><span><a href="#Results/Conclusions" data-toc-modified-id="Results/Conclusions-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Results/Conclusions</a></span><ul class="toc-item"><li><span><a href="#Further-Exploration/Questions" data-toc-modified-id="Further-Exploration/Questions-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Further Exploration/Questions</a></span></li></ul></li></ul></li></ul></div>

insert header

# Title

## Business Understanding and Business Problem

### Cost of Errors

## Data Understanding

In [1]:
#imports grabbed from lecture
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

### Initial Data

In [2]:
#the four csv files provided
links_df = pd.read_csv('data/links.csv')
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')
tags_df = pd.read_csv('data/tags.csv')

In [3]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [39]:
movies_df.shape

(9742, 3)

In [41]:
movies_df['title'].nunique()

9737

In [43]:
movies_df['title'].value_counts().head(10)

Emma (1996)                                   2
Saturn 3 (1980)                               2
Eros (2004)                                   2
Confessions of a Dangerous Mind (2002)        2
War of the Worlds (2005)                      2
Campus Man (1987)                             1
Land of the Dead (2005)                       1
Texas Chainsaw Massacre, The (2003)           1
Carabineers, The (Carabiniers, Les) (1963)    1
Mr. Popper's Penguins (2011)                  1
Name: title, dtype: int64

There are 9737 unique movie titles. Repeats for 5 movies:
- Emma (1996)                                   2
- Saturn 3 (1980)                               2
- Eros (2004)                                   2
- Confessions of a Dangerous Mind (2002)        2
- War of the Worlds (2005)                      2

In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [17]:
ratings_df.groupby('movieId').count()

,userId,rating,timestamp
movieId,,,
1,215,215,215
2,110,110,110
3,52,52,52
4,7,7,7
5,49,49,49
...,...,...,...
193581,1,1,1
193583,1,1,1
193585,1,1,1


In [25]:
new_ratings_df = ratings_df.drop('timestamp', axis=1).copy()

In [26]:
merged_df = new_ratings_df.merge(movies_df, how='outer', on='movieId')

In [32]:
merged_df.isna().sum()

userId     18
movieId     0
rating     18
title       0
genres      0
dtype: int64

In [33]:
merged_df.tail(20)

,userId,movieId,rating,title,genres
100834,610.0,163937,3.5,Blair Witch (2016),Horror|Thriller
100835,610.0,163981,3.5,31 (2016),Horror
100836,NaN,1076,NaN,"Innocents, The (1961)",Drama|Horror|Thriller
100837,NaN,2939,NaN,Niagara (1953),Drama|Thriller
100838,NaN,3338,NaN,For All Mankind (1989),Documentary
100839,NaN,3456,NaN,"Color of Paradise, The (Rang-e khoda) (1999)",Drama
100840,NaN,4194,NaN,I Know Where I'm Going! (1945),Drama|Romance|War
100841,NaN,5721,NaN,"Chosen, The (1981)",Drama
100842,NaN,6668,NaN,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
100843,NaN,6849,NaN,Scrooge (1970),Drama|Fantasy|Musical


In [34]:
no_nulls = merged_df.dropna()

In [35]:
no_nulls

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610.0,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610.0,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610.0,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610.0,163937,3.5,Blair Witch (2016),Horror|Thriller


In [46]:
pivot_test = no_nulls.pivot_table(index='userId', columns='title', values='rating')

In [47]:
pivot_test.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Setting up basic model?

In [44]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [56]:
no_nulls_test = no_nulls.drop(['movieId', 'genres'], axis=1).head()

In [57]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(no_nulls_test,reader)

ValueError: could not convert string to float: 'Toy Story (1995)'

In [49]:
sim_cos = {'name':'cosine', 'user_based':False}

In [51]:
basic = knns.KNNBasic()
basic.fit(pivot_test)

AttributeError: 'DataFrame' object has no attribute 'n_users'

## Exploratory Data Analysis

## Modeling

## Final Model

## Results/Conclusions

### Further Exploration/Questions